In [1]:
library(RODBC)
library(stats)
library(ggplot2)
library(tidyr)

library(repr)


In [2]:
odb = odbcConnect('dev mit ro')
db = sqlQuery(odb, "select a.*, b.asg * b.asg as asg2_1yr, c.asg*c.asg as asg2_emp
                    from clean_calc.output_emp_vol_aret a 
                    left join clean_calc.output_emp_vol_1yr b
                    on a.pid = b.pid and a.yearmon = b.yearmon - 100
                    left join clean_calc.output_emp_vol c
                    on a.pid = c.pid and a.yearmon = c.yearmon
                    where b.asg is not null and b.asg<>0
                    and a.yearmon>199000 and a.yearmon<200000
                    --where a.yearmon = 199501 --and b.asg is not null and b.asg<>0
              ")
odbcClose(odb)

In [ ]:
odbcClose(odb)

In [ ]:
#save(db, file = "db_19901999.RData")

In [ ]:
names(db)[3:63] = paste0('m_', names(db)[3:63])
db <- select(db, -60)
copy = db

In [ ]:
db <- select(db, c("pid", "yearmon", "asg2_1yr", paste0('m_', seq(0,29))))
db <- drop_na(db)

print(paste("Number of records from source ", format(dim(copy)[1], big.mark=","))
print(paste("Number of records in current db ", format(dim(db)[1], big.mark=","))
print(paste("Error asg2_1yr and row_mean: ", sqrt(sqrt(mean((db$asg2_1yr - db$row_mean)^2))))
print(paste("Error asg2_emp and row_mean: ", sqrt(sqrt(mean((db$asg2_emp - db$row_mean)^2))))

In [ ]:
for(lag in seq(3, 30, by = 3)){
  # lag = 6
  model1 = lm(db, formula = paste('asg2_1yr ~ -1', paste(paste0(' + m_', seq(0, lag - 1)), collapse = '')))
  summary(model1)
  error = sqrt(sqrt(mean((db$asg2_1yr - predict(model1, db))^2, na.rm = T)))
  print(paste0("lag=", lag, ", error=", error))
  error_orig = sqrt(sqrt(mean((db$asg2_1yr - db$asg ^ 2)^2, na.rm = T)))
  db$pred = predict(model1, db)
  p = ggplot(db, aes(x = sqrt(asg2_1yr), y = sqrt(pred))) + geom_point() + geom_abline(intercept = 0, slope = 1, lty = 2) + 
    scale_x_log10() + scale_y_log10() + labs(title = paste0('lags = ', lag, ', error = ', scales::percent(error)))
  #print(p)
}

In [ ]:
get_gamma <- function(beta){
  alpha = log((1-exp(beta)) / (exp(beta)-exp(beta*(lag+1))))
  return(t(matrix(exp(alpha + beta*seq(1,lag)))))
}

get_prediction <- function(beta) {
  gamma = get_gamma(beta)
  x = db[,paste(paste0('m_', seq(0, lag - 1)))]
  prediction =  gamma%*%t(as.matrix(x))
}

min_f <-function(beta){
  x = db[,paste(paste0('m_', seq(0, lag - 1)))]
  prediction =  get_prediction(beta)
  error = sum((db$asg2_1yr - prediction)^2, na.rm=T)
  return(error)
}

get_error <- function(beta){
  x = db[,paste(paste0('m_', seq(0, lag - 1)))]
  prediction =  get_prediction(beta)
  error = mean((db$asg2_1yr - prediction)^2, na.rm=T)
  return(sqrt(sqrt(error)))
}

In [ ]:
for (lag in seq(3,30, by=3)){
  model = nlm(min_f,c(-2))
  error =get_error(model$estimate)
  print(paste0("lag:", lag, " error=", error))
}